In [0]:
#importerer nødvendige bibloteker
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [4]:
#batch size  er antall training eks i brukt i en iterasjon
BATCH_SIZE= 500

#kilde til kode fra lab7. 
#transformations
transform = transforms.Compose(
    [transforms.ToTensor()])

# laster ned og loader training dataset
train_set = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
test_set = torchvision.datasets.FashionMNIST(root="./data", download=True, train= False, transform=
                                              transforms.Compose([transforms.ToTensor()]))
trainloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


In [0]:
#Oppg1: NN med to conv
class FashionNN(nn.Module):
    def __init__(self):
        super(FashionNN, self).__init__()

        # Definere conventional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        #fully connected layers
        self.fc1 = nn.Linear(26 * 26 * 32, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, t):
       
        t = self.conv1(t)
        t = F.relu(t)
        #relu er activation funksjon

        #flatten, for å skape en single long feature vector
        t = t.flatten(start_dim = 1)


        t = self.fc1(t)
        t = F.relu(t)


        logits = self.fc2(t)
        #softmax er en aktivering funksjon som forandre tall til sanns for summen av en. 
        out = F.softmax(logits, dim=1)
        return out

In [7]:
learning_rate = 0.001
num_epochs = 6

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = FashionNN()
model = model.to(device)

#crossentropyloss, innebygd funksjon som måler 
#performancene av modelen output med sanns verdi mellom 1  og 0
error = nn.CrossEntropyLoss()
# adam er en innbydg opmtimizer algoritme
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)



FashionNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=21632, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [8]:
#funksjon for accurancy
def get_acc(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()


#Oppg2: train nn.
for epoch in range(num_epochs):
    loss_train = 0.0
    accuracy_train = 0.0

    model = model.train()

    ## training steg
    for i, (images, labels) in  enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)
        loss = error(logits, labels)
        # vitkig at disse brukes i rekkefølgen, for å unnga at man akkumulerer 
        #gradient fra multi passes
        
        #zero_grad for å clear gamle gradients
        optimizer.zero_grad()
        
        #derivere tapet for parameter som er true. 
        loss.backward()
        
        #oppdaterer verdien av x basert på the current gradient
        optimizer.step()

        loss_train += loss.detach().item()
        accuracy_train += get_acc(logits, labels, BATCH_SIZE)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, loss_train / i, accuracy_train/i))

Epoch: 0 | Loss: 1.7393 | Train Accuracy: 75.31
Epoch: 1 | Loss: 1.6368 | Train Accuracy: 84.95
Epoch: 2 | Loss: 1.6184 | Train Accuracy: 86.73
Epoch: 3 | Loss: 1.6057 | Train Accuracy: 87.94
Epoch: 4 | Loss: 1.5965 | Train Accuracy: 88.85
Epoch: 5 | Loss: 1.5899 | Train Accuracy: 89.52


In [9]:
#Test accuracy
test_accuracy = 0.0
for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_accuracy += get_acc(outputs, labels, BATCH_SIZE)
        
print('Test Accuracy: %.2f'%( test_accuracy/(i+1)))

Test Accuracy: 87.94


In [17]:
#Lagre 
model= FashionNN()
torch.save(model, 'Oblig2.pt')




/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type FashionNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [11]:
len(train_set.targets)

60000

In [12]:
#confusion matrix
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

test_acc= test_accuracy/(i+1)
acc= test_acc/len(train_set.targets)
print(acc)





#confusion_matrix(labels, predictions)
#utregning
#print("Classification report for CNN :\n%s\n"
      #% (metrics.classification_report(labels, predictions)))

0.0014656667391459146
